Zooi importen

In [ ]:
# basic libraries
import numpy as np

# computational geometry libraries
import pyvista as pv
from compas.datastructures import Mesh as CompasMesh
from compas_plotters.meshplotter import MeshPlotter

# libraries for connection to grasshopper (speckle)
from specklepy.api.client import SpeckleClient
from specklepy.api.credentials import get_default_account
from specklepy.transports.server import ServerTransport
from specklepy.api import operations

# Earthy Bridge
import earthy_bridge as eb

# extra IPython utilities
from IPython.display import display

Verbinding maken met speckle

In [ ]:
# initialise the client
client = SpeckleClient(host="speckle.xyz", use_ssl=True)

# authenticate the client with a token
account = get_default_account()
client.authenticate(token=account.token)

# use that stream id to get the stream from the server
mesh_stream = client.stream.get(id="481d561078")

# creating a server transport 
transport = ServerTransport(client=client, stream_id=mesh_stream.id)

Import mesh from speckle

In [ ]:
# find base box reference id
last_geo_ref = client.commit.list(mesh_stream.id)[0].referencedObject

# this receives the object back from the transport.
received_mesh = operations.receive(obj_id=last_geo_ref, remote_transport=transport)

# visualize the mesh
#eb.speckle_to_pyvista(received_mesh).plot()

In [ ]:
# construct the compas mesh
(V, F) = eb.speckle_to_vertices_and_faces(received_mesh)

In [ ]:
# construct the compas mesh
received_mesh_compas = CompasMesh.from_vertices_and_faces(V,F)

In [ ]:
dual_mesh = received_mesh_compas.dual()
dV, dF = dual_mesh.to_vertices_and_faces()

In [ ]:
p = pv.Plotter()
p.add_mesh(eb.compas_to_pyvista(dual_mesh))
p.show()

Export to grashopper

In [ ]:
# construct speckle mesh
edited_mesh_speckle = eb.compas_to_speckle(dual_mesh)

In [ ]:
# Serialises the block and sends it to the transport
edited_mesh_hash = operations.send(base=edited_mesh_speckle, transports=[transport])

# creating a commit on your stream with this object
commid_id = client.commit.create(
    stream_id=mesh_stream.id, 
    object_id=edited_mesh_hash, 
    message="edited_mesh")

In [11]:
dual_mesh.to_obj("data/dual_mesh.obj")